In [9]:
import os
import random
import uuid
import numpy as np
from faker import Faker
from datetime import datetime
import time
import boto3
import json
from dotenv import load_dotenv

In [10]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID_AWS')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY_AWS')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION_AWS')

In [11]:
# Setup boto3 client with real credentials
kinesis_client = boto3.client('kinesis', 
                              aws_access_key_id=AWS_ACCESS_KEY_ID,
                              aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                              region_name=AWS_REGION)

In [12]:
def send_to_kinesis(order, stream_name):
    kinesis_client.put_record(
        StreamName=stream_name,
        Data=json.dumps(order),
        PartitionKey=order["order_id"]
    )

In [13]:
# Initialize Faker
fake = Faker('es-ES')

# Constants for random address generation in Madrid
LAT_MIN, LAT_MAX = 40.3121, 40.6521
LON_MIN, LON_MAX = -3.8666, -3.5673
max_vol_per_order = 500.0
max_weight_per_order = 100.0
status = "RECEIVED"

stream_name = 'OrderStreamForDispatching'  # Update with your Kinesis stream name

In [14]:
# Function to generate a random address in Madrid
def generate_random_address():
    street_name = fake.street_name()
    street_number = fake.building_number()
    lat = random.uniform(LAT_MIN, LAT_MAX)
    lon = random.uniform(LON_MIN, LON_MAX)
    address = f"{street_name}, {street_number}, Madrid, Spain"
    return {
        "address": address,
        "lat": lat,
        "lon": lon
    }

# Function to generate a random order
def generate_random_order(client_id):
    order_id = str(uuid.uuid4())
    volume = np.random.uniform(1, max_vol_per_order)
    weight = np.random.uniform(1, max_weight_per_order)
    n_objects = np.random.randint(1, 10)
    
    # Ensure the timestamp is generated in the correct format (Y-M-D H:M:S)
    order_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Ensure the total price is a random float and never 0
    total_price = np.random.uniform(10.0, 1000.0)
    
    order = {
        "order_id": order_id,
        "customer_id": f"cus-{uuid.uuid4()}",  # Simulating customer_id format
        "total_weight": weight,
        "total_volume": volume,
        "total_price": total_price,
        "order_timestamp": order_timestamp,  # Formatted timestamp
        "status": status,
        "lat": random.uniform(LAT_MIN, LAT_MAX),
        "lon": random.uniform(LON_MIN, LON_MAX)
    }
    print("Generated order:", order)  # Print the generated order for debugging
    return order

In [15]:
# Function to send the generated order to Kinesis
def send_order_to_kinesis(stream_name, order):
    order_data = json.dumps(order)
    
    response = kinesis_client.put_record(
        StreamName=stream_name,
        Data=order_data.encode('utf-8'),
        PartitionKey=str(order['customer_id'])  # Partition based on customer_id
    )
    print(f"Order {order['order_id']} sent to Kinesis. Response: {response}")

# Function to generate orders at random intervals and send them to Kinesis
def order_stream_generator(stream_name):
    while True:
        client_id = random.randint(100000000000, 999999999999)
        order = generate_random_order(client_id)
        send_order_to_kinesis(stream_name, order)
        
        # Sleep for a random amount of time between 10 and 50 seconds (can be adjusted)
        sleep_time = random.randint(10, 30)  # Random sleep time in seconds
        print(f"Sleeping for {sleep_time / 60} minutes")
        time.sleep(sleep_time)


In [16]:
# Start the order stream generator
order_stream_generator(stream_name)

Generated order: {'order_id': 'c7cf890d-de87-4a5a-9240-3b36ef0585fa', 'customer_id': 'cus-bb1812de-2ce5-4845-8a4f-a9e89cffca15', 'total_weight': 71.35465825237074, 'total_volume': 439.7739130906643, 'total_price': 286.63515411509513, 'order_timestamp': '2024-10-08 18:24:45', 'status': 'RECEIVED', 'lat': 40.343698265769135, 'lon': -3.6269222039103233}
Order c7cf890d-de87-4a5a-9240-3b36ef0585fa sent to Kinesis. Response: {'ShardId': 'shardId-000000000000', 'SequenceNumber': '49656572502580504910622657994153777067251454247109132290', 'ResponseMetadata': {'RequestId': 'f0c81ed4-9f12-aad3-afb7-41d935c3b6cf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f0c81ed4-9f12-aad3-afb7-41d935c3b6cf', 'x-amz-id-2': '8iwZRZEJtMUe3pLHtGwEnOv5uduZwiBZ1c6rLhEr8ewPfqWC4oH72/fobDZvm4CT7+A5MHeN0a5OnVGVTjRMxEJ4wskswrE7', 'date': 'Tue, 08 Oct 2024 16:24:45 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
Sleeping for 0

KeyboardInterrupt: 